# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC

#set random state
RANDOM_STATE = 42


In [ ]:
# load data from database
engine = create_engine('sqlite:///messages.db')
df = pd.read_sql("SELECT * FROM messages", engine)

In [ ]:
df.head()

In [ ]:
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):

    #Remove punctuation and convert to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #Tokenize
    tokens = word_tokenize(text)
    
    #Stem words
    stemmer = PorterStemmer()
    
    #Remove stop words
    stop_words_english = stopwords.words("english")
    
    stemmed = [stemmer.stem(word) for word in tokens if word not in stop_words_english]
    
    return stemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [48]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [49]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = RANDOM_STATE)

np.random.seed(RANDOM_STATE)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [50]:
def get_eval_metrics(actual, predicted, col_names):

    metrics = []
    
    # Calculate evaluation metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(actual[:, i], predicted[:, i])
        precision = precision_score(actual[:, i], predicted[:, i], average = 'weighted')
        recall = recall_score(actual[:, i], predicted[:, i], average = 'weighted')
        f1 = f1_score(actual[:, i], predicted[:, i], average = 'weighted')
        
        metrics.append([accuracy, precision, recall, f1])
    
    # Create dataframe containing metrics
    metrics = np.array(metrics)
    metrics_df = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return metrics_df

In [51]:
Y_train_pred = pipeline.predict(X_train)
col_names = list(Y.columns.values)

print(get_eval_metrics(np.array(Y_train), Y_train_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.989523   0.989502  0.989523  0.989492
request                 0.987489   0.987600  0.987489  0.987319
offer                   0.998830   0.998832  0.998830  0.998739
aid_related             0.986166   0.986282  0.986166  0.986144
medical_help            0.988099   0.988251  0.988099  0.987651
medical_products        0.991405   0.991470  0.991405  0.991019
search_and_rescue       0.994304   0.994324  0.994304  0.994005
security                0.996033   0.996034  0.996033  0.995799
military                0.994863   0.994881  0.994863  0.994662
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.995982   0.995999  0.995982  0.995921
food                    0.994711   0.994732  0.994711  0.994657
shelter                 0.992422   0.992478  0.992422  0.992270
clothing                0.997304   0.997301  0.997304  0.997184
money                   0.995728   0.995

In [52]:
Y_test_pred = pipeline.predict(X_test)

print(get_eval_metrics(np.array(Y_test), Y_test_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.798444   0.783640  0.798444  0.785854
request                 0.884498   0.876817  0.884498  0.869840
offer                   0.994965   0.989955  0.994965  0.992454
aid_related             0.748703   0.746704  0.748703  0.744223
medical_help            0.922032   0.902422  0.922032  0.895248
medical_products        0.949344   0.937343  0.949344  0.928704
search_and_rescue       0.975893   0.966536  0.975893  0.967239
security                0.981538   0.967575  0.981538  0.973435
military                0.969789   0.958416  0.969789  0.958824
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.949191   0.942809  0.949191  0.937164
food                    0.936832   0.932604  0.936832  0.931130
shelter                 0.931340   0.922983  0.931340  0.917749
clothing                0.985810   0.982373  0.985810  0.980195
money                   0.980775   0.981

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [53]:
clf_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__min_samples_split': [2, 3, 4]
}

grid_search = GridSearchCV(pipeline, param_grid = clf_parameters, cv = 5, verbose = 2, n_jobs=-1)

In [ ]:
grid_search.fit(X_train, Y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__ngram_range=(1, 1) 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__ngram_range=(1, 1) 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__ngram_range=(1, 1) 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__ngram_range=(1, 1) 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__ngram_range=(1, 1) 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__ngram_range=(1, 2) 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__ngram_range=(1, 2) 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__us

In [ ]:
best_params = grid_search.best_params_

In [ ]:
best_estimator = grid_search.best_estimator_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
Y_pred_tuned = best_estimator.predict(X_test)

print((np.array(Y_test), Y_pred_tuned, col_names))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

We will include both the word and its n-grams in the pipeline and use a linear SVC model

In [ ]:
# Define a custom transformer to extract character n-grams
class CharacterNGramExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, ngram_range=(2, 5)):
        self.ngram_range = ngram_range

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        char_vectorizer = CountVectorizer(analyzer='char', ngram_range=self.ngram_range)
        return char_vectorizer.fit_transform(X)

# Create a new pipeline
pipeline_improved = Pipeline([
    ('features', FeatureUnion([
        ('word_ngrams', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('char_ngrams', Pipeline([
            ('char_vect', CharacterNGramExtractor()),
            ('tfidf', TfidfTransformer())
        ]))
    ])),
    ('clf', MultiOutputClassifier(LinearSVC()))
])

# Define the parameter grid for GridSearchCV
param_grid = {
    'features__word_ngrams__vect__ngram_range': [(1, 1), (1, 2)],
    'features__word_ngrams__tfidf__use_idf': (True, False),
    'features__char_ngrams__char_vect__ngram_range': [(2, 4), (2, 5)],
    'clf__estimator__C': [0.1, 1, 10],
}

# Create the GridSearchCV instance with cross-validation settings
grid_search_improved = GridSearchCV(pipeline_improved, param_grid, cv=5, verbose=2, n_jobs=-1)


In [ ]:
grid_search_improved.fit(X_train, y_train)

In [ ]:
best_params_improved = grid_search_improved.best_params_

In [ ]:
best_estimator_improved = grid_search_improved.best_estimator_

In [ ]:
Y_pred_improved = best_estimator_improved.predict(X_test)

print((np.array(Y_test), Y_pred_improved, col_names))

### 9. Export your model as a pickle file

In [ ]:
with open('best_model.pkl', 'wb') as file:
    pickle.dump(best_estimator_improved, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.